In [1]:
from query_dfs import get_dfs

In [2]:
df_orders, df_train, df_prior = get_dfs()

In [6]:
df_orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1902925,611,prior,6,5,22,6.0
1,930909,611,prior,7,5,19,7.0
2,2699188,611,prior,8,5,23,21.0
3,2548905,611,prior,9,4,19,30.0
4,1616486,611,prior,10,6,10,9.0


In [7]:
df_train.head(5)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,6129,24852,1,1,38907,train,7,1,14,30.0
1,6129,48364,2,1,38907,train,7,1,14,30.0
2,6129,21903,3,1,38907,train,7,1,14,30.0
3,6129,7806,4,1,38907,train,7,1,14,30.0
4,6129,27966,5,1,38907,train,7,1,14,30.0


In [8]:
df_prior.head(5)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,114,24954,1,0,91891,prior,1,0,11,NaN
1,114,1688,2,0,91891,prior,1,0,11,NaN
2,114,37371,3,0,91891,prior,1,0,11,NaN
3,114,5782,4,0,91891,prior,1,0,11,NaN
4,114,1263,5,0,91891,prior,1,0,11,NaN
